In [7]:
# skip copying of files if they already exist in the repo data-scenario folder
default_overwrite = False

base_output_dir = "data-scenario"

files_to_copy = [
    "4_ModeChoice/1a_Skims/skm_w8_Pk.omx",
    "4_ModeChoice/1a_Skims/skm_w8_Ok.omx",
    "4_ModeChoice/1a_Skims/skm_d8_Pk.omx",
    "4_ModeChoice/1a_Skims/skm_d8_Ok.omx",
    "4_ModeChoice/2_DetailedTripMatrices/HBC_trips_allsegs_Pk.omx ",
    "4_ModeChoice/2_DetailedTripMatrices/NHB_trips_allsegs_Ok.omx ",
    "4_ModeChoice/2_DetailedTripMatrices/HBO_trips_allsegs_Ok.omx ",
    "4_ModeChoice/2_DetailedTripMatrices/HBW_trips_allsegs_Ok.omx",
    "4_ModeChoice/2_DetailedTripMatrices/HBW_trips_allsegs_Pk.omx ",
    "4_ModeChoice/2_DetailedTripMatrices/NHB_trips_allsegs_Pk.omx ",
    "4_ModeChoice/2_DetailedTripMatrices/HBO_trips_allsegs_Pk.omx",
    "4_ModeChoice/3_TransitAssign/%RUNID%_transit_brding_summary_node.csv",
    "4_ModeChoice/3_TransitAssign/%RUNID%_transit_rider_summary_link.csv"
]

In [8]:
# COPY DATA FILES FROM MODEL FOLDERS TO data-scenario FOLDER
# This script reads a YAML file containing scenarios and models, and copies specified files from each model's folder to a common output directory.
# It also generates two CSV files: one for scenarios and one for models, containing relevant metadata.
# The script handles file overwriting based on a specified default behavior and allows for dynamic RUNID replacement in file paths.

import yaml
import pandas as pd
import os
import shutil

# Load YAML directly as a list
with open("_scenarios.yaml", "r") as f:
    data = yaml.safe_load(f)

# Normalize into a flat structure: one row per model per scenario
rows_models = []
rows_scenarios = []
for scenario in data:
    rows_scenarios.append({
        "scenario_id": scenario["id"],
        "scenario_name": scenario["name"],
        "scenario_folder": scenario["folder"]
    })
    print(f"Scenario: {scenario['name']} ({scenario['id']})")
    for model in scenario["models"]:
        model_RUNID = model["RID"] if "RID" in model else None
        overwrite = model["overwrite"] if "overwrite" in model else default_overwrite
        rows_models.append({
            "model_id": scenario["id"] + '__' + model["name"],
            "model_RID": model_RUNID,
            "model_name": model["name"],
            "model_overwrite": overwrite,
            "model_display": model["display"],
            "model_folder": scenario['folder'] + '\\' + model['name'],
            "model_label": scenario['name'] + ' - ' + model['name'].replace('_',' '),
            "scenario_id": scenario["id"]
        })

        target_folder = os.path.join(base_output_dir, scenario["id"], model["name"])
        os.makedirs(target_folder, exist_ok=True)

        for rel_path in files_to_copy:
            # Replace __RUNID__ in source path
            source_path = os.path.join(scenario["folder"], model["name"], rel_path.replace("%RUNID%", model_RUNID))
            filename = os.path.basename(source_path).replace(model_RUNID, "").lstrip('_') # remove RUNID from filename

            # Save to target without subfolders, with __RUNID__ retained if it was used
            target_path = os.path.join(target_folder, filename)


            if os.path.exists(target_path) and not overwrite:
                #print(f"⏭️ Skipped (exists, no overwrite): {target_path}")
                continue

            else:
                if not os.path.exists(source_path):
                    print(f"⚠️ File not found: {source_path}")
                    continue

            shutil.copy2(source_path, target_path)
            print(f"✅ Copied {source_path} -> {target_path}")

models_df = pd.DataFrame(rows_models)
display(models_df)
models_df.to_csv(f'{base_output_dir}/models.csv', index=False)
scenarios_df = pd.DataFrame(rows_scenarios)
display(scenarios_df)
scenarios_df.to_csv(f'{base_output_dir}/scenarios.csv', index=False)
print("Done!")

Scenario: v9.1-official (v910-official)
Scenario: Recalib-RF2.5 (E2.14.4)
Scenario: Utah County MaxTime 15 (E2.16.2)
Scenario: All Counties MaxTime 15 (E2.16.3)
Scenario: UTCo dCRT Adj -15 (E2.17.3)
Scenario: UTCo dCRT Adj -15 + SoUTCo dCRT Adj -60 Start in Dest Choice (E2.18.2)
Scenario: UTCo dCRT Adj -15 + SoUTCo dCRT Adj -45 Start in Dest Choice (E2.18.3)


,model_id,model_RID,model_name,model_overwrite,model_display,model_folder,model_label,scenario_id
0,v910-official__BY_2019,WFv910_BY_2019,BY_2019,False,True,E:\GitHub\WF-TDM-v9x\Scenarios\v910-official\B...,v9.1-official - BY 2019,v910-official
1,v910-official__RTP_2050,WFv910_RTP_2050,RTP_2050,False,True,E:\GitHub\WF-TDM-v9x\Scenarios\v910-official\R...,v9.1-official - RTP 2050,v910-official
2,E2.14.4__BY_2019,WFv920-E2.14.4_BY_2019,BY_2019,False,True,E:\GitHub\WF-TDM-v9x\Scenarios\v920-E2.14.4\BY...,Recalib-RF2.5 - BY 2019,E2.14.4
3,E2.16.2__BY_2019,WFv920-E2.16.2_BY_2019,BY_2019,False,True,E:\GitHub\WF-TDM-v9x\Scenarios\v920-E2.16.2\BY...,Utah County MaxTime 15 - BY 2019,E2.16.2
4,E2.16.3__BY_2019,WFv920-E2.16.3_BY_2019,BY_2019,False,True,E:\GitHub\WF-TDM-v9x\Scenarios\v920-E2.16.3\BY...,All Counties MaxTime 15 - BY 2019,E2.16.3
5,E2.17.3__BY_2019,WFv920-E2.17.3_BY_2019,BY_2019,False,True,E:\GitHub\WF-TDM-v9x\Scenarios\v920-E2.17.3\BY...,UTCo dCRT Adj -15 - BY 2019,E2.17.3
6,E2.18.2__BY_2019,WFv920-E2.18.2_BY_2019,BY_2019,False,True,E:\GitHub\WF-TDM-v9x\Scenarios\v920-E2.18.2\BY...,UTCo dCRT Adj -15 + SoUTCo dCRT Adj -60 Start ...,E2.18.2
7,E2.18.2__RTP_2050,WFv920-E2.18.2_RTP_2050,RTP_2050,False,True,E:\GitHub\WF-TDM-v9x\Scenarios\v920-E2.18.2\RT...,UTCo dCRT Adj -15 + SoUTCo dCRT Adj -60 Start ...,E2.18.2
8,E2.18.2__RTP_MedDistAdj_2050,WFv920-E2.18.2_RTP_MedDistAdj_2050,RTP_MedDistAdj_2050,False,True,E:\GitHub\WF-TDM-v9x\Scenarios\v920-E2.18.2\RT...,UTCo dCRT Adj -15 + SoUTCo dCRT Adj -60 Start ...,E2.18.2
9,E2.18.3__BY_2019,WFv920-E2.18.3_BY_2019,BY_2019,False,True,E:\GitHub\WF-TDM-v9x\Scenarios\v920-E2.18.3\BY...,UTCo dCRT Adj -15 + SoUTCo dCRT Adj -45 Start ...,E2.18.3


,scenario_id,scenario_name,scenario_folder
0,v910-official,v9.1-official,E:\GitHub\WF-TDM-v9x\Scenarios\v910-official
1,E2.14.4,Recalib-RF2.5,E:\GitHub\WF-TDM-v9x\Scenarios\v920-E2.14.4
2,E2.16.2,Utah County MaxTime 15,E:\GitHub\WF-TDM-v9x\Scenarios\v920-E2.16.2
3,E2.16.3,All Counties MaxTime 15,E:\GitHub\WF-TDM-v9x\Scenarios\v920-E2.16.3
4,E2.17.3,UTCo dCRT Adj -15,E:\GitHub\WF-TDM-v9x\Scenarios\v920-E2.17.3
5,E2.18.2,UTCo dCRT Adj -15 + SoUTCo dCRT Adj -60 Start ...,E:\GitHub\WF-TDM-v9x\Scenarios\v920-E2.18.2
6,E2.18.3,UTCo dCRT Adj -15 + SoUTCo dCRT Adj -45 Start ...,E:\GitHub\WF-TDM-v9x\Scenarios\v920-E2.18.3


Done!
